### Построение рекомендаций на основе популярности товаров

**Задача**: скачать датасет DIGINETICA и написать код, который строит рекомендации к каждому товару на основании популярности. На выходе должен получиться CSV-файл формата:

```
seed_item_id, recommended_product_id, score

```

### Ожидания:

- Напишите код, который:
    - Загружает датасет.
    - Рассчитывает популярность товаров (например, на основании количества просмотров или покупок).
    - Создает рекомендации для каждого товара.
    - Сохраняет результат в CSV.

### Реализация:

Будем оценивать популярность на основе статистики покупок.

Для этого исследуем датасет 'train-purchases.csv'

In [19]:
import pandas as pd

In [20]:
# Загрузка данных
data  = pd.read_csv('train-purchases.csv', sep=';')
data .head()

,sessionId,userId,timeframe,eventdate,ordernumber,itemId
0,150,18278.0,17100868,2016-05-06,16421,25911
1,151,NaN,6454547,2016-05-06,16290,175874
2,156,7.0,1721689387,2016-05-27,21173,35324
3,179,NaN,343001,2016-05-09,16924,31233
4,246,34.0,2311046,2016-05-09,16936,34677


In [21]:
data.shape

(18025, 6)

In [22]:
# Удаление дубликатов
data = data.drop_duplicates()
data.shape

(17997, 6)

In [23]:
# Рассчет популярности товаров (на основании встречаемости itemId в данных)
item_popularity = data.groupby('itemId').size().reset_index(name='popularity')

# Сортировка товаров по популярности
item_popularity = item_popularity.sort_values(by='popularity', ascending=False)
item_popularity.head()

,itemId,popularity
1883,11338,28
1806,10858,28
6772,79141,22
44,332,20
437,2809,20


In [24]:
# Создаем рекомендации для каждого товара
N = 5  # Количество товаров для рекомендации
recommendations = []

for item_id in item_popularity['itemId']:
    top_items = item_popularity[item_popularity['itemId'] != item_id].head(N)
    for _, row in top_items.iterrows():
        recommendations.append([item_id, row['itemId'], row['popularity']])

In [25]:
# Преобразуем рекомендации в DataFrame
recommendations_df = pd.DataFrame(recommendations, columns=['seed_item_id', 'recommended_product_id', 'score'])

In [26]:
# Сохраняем результат в CSV
recommendations_df.to_csv('recommendations.csv', index=False)

In [27]:
# Проверим получившийся файл
recs  = pd.read_csv('recommendations.csv')
recs .head()

,seed_item_id,recommended_product_id,score
0,11338,10858,28
1,11338,79141,22
2,11338,332,20
3,11338,2809,20
4,11338,72786,18
